__ Naive Bayer Classifier __

In [28]:
from pandas import read_csv
#train_test_split
#GridSearchCV for cross validation
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB

df = read_csv("./train.csv",delimiter=";")

df.head(5)

,A,B,C,D,E,F,G,H,I,J,...,AF,AG,AH,AI,AJ,AK,AL,AM,AN,Target
0,0.299403,-1.226624,1.498425,-1.176150,5.289853,0.208297,2.404498,1.594506,-0.051608,0.663234,...,-0.622990,-1.833057,0.293024,3.552681,0.717611,3.305972,-2.715559,-2.682409,0.101050,1
1,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761,0
2,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436,0
3,1.573270,-0.580318,-0.866332,-0.603812,3.125716,0.870321,-0.161992,4.499666,1.038741,-1.092716,...,1.275598,-3.480110,-1.065252,2.153133,1.563539,2.767117,0.215748,0.619645,1.883397,1
4,-0.613071,-0.644204,1.112558,-0.032397,3.490142,-0.011935,1.443521,-4.290282,-1.761308,0.807652,...,-1.803473,0.518579,-0.205029,-4.744566,-1.520015,1.830651,0.870772,-1.894609,0.408332,0


**Splitting the dataframe to X and Y**

In [29]:
#drop target cols
X = df.drop('Target',axis=1)
#split target cols
y = df.Target

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
X_train.shape, X_test.shape

((800, 40), (200, 40))

**Train Naive Bayes**

In [32]:
NB = GaussianNB()
NB.fit(X_train,y_train)

GaussianNB(priors=None)

**Get Accuracy of NB**

In [33]:
NB.score(X_test,y_test)

0.82

**Example using GridSearchCV to test with multiple hyperparameter tuning**

Cross validation should be implemented to training data only

In [34]:
parameters = {
    "alpha" : [0.2, 0.5, 1],
    "fit_prior" : [True,False]
}
cross_validate_folds = 3
NBwithGridSearch = GridSearchCV(BernoulliNB(),param_grid=parameters,cv=cross_validate_folds)
NBwithGridSearch.fit(X_train,y_train)
NBwithGridSearch.best_score_

0.75375

In [35]:
#finding the best params of hyperparameters configuration
NBwithGridSearch.best_params_

{'alpha': 0.2, 'fit_prior': False}

In [36]:
y_test = NBwithGridSearch.predict(X_test)

**try data preprocessing and pipeline**

In [37]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

In [42]:
steps = Pipeline([
    ("poly", PolynomialFeatures()),
    ("scaler", StandardScaler()),
    ("NB",BernoulliNB()),
])
'''
key should be followed the described key in the pipeline followed by __ (2 underscores) e.g. poly__degree : [2]
if enabled, it enable the interaction between params, can be params1 multiplied by params2 
'''
parameters = {
    "poly__degree" : [1,2],
    "poly__interaction_only" : [True,False], 
    "NB__alpha" : [0.2, 0.5, 1],
    "NB__fit_prior" : [True, False]
}

NBWithPipeline = GridSearchCV(steps,param_grid=parameters,cv=3, scoring="f1")
NBWithPipeline.fit(X_train, y_train)
NBWithPipeline.best_score_

0.7673665444770283

In [43]:
NBWithPipeline.best_params_

{'NB__alpha': 0.2,
 'NB__fit_prior': True,
 'poly__degree': 1,
 'poly__interaction_only': True}

In [44]:
NBWithPipeline.score(X_train,y_train), NBWithPipeline.score(X_test, y_test)

(0.7722289890377588, 0.900990099009901)